In [29]:
from preamble import *
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('housing.csv')
df['ocean_proximity'].value_counts()

ocean_proximity
<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: count, dtype: int64

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [17]:
204/20640

0.009883720930232558

In [ ]:
df.dropna() #207개 삭제 0.01% 정도여서 그냥 일단 삭제

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


## 데이터 전처리

In [18]:
## x와 y

X = df.drop("median_house_value", axis=1)
y = df['median_house_value']

## 수치형과 범주형으로 데이터를 나눠보세요(리스트로 만드세요!)
# num_features = X.drop("ocean_proximity")
# cat_features = X['ocean_proximity']

## 수치형과 범주형으로 데이터를 나눠보세요(리스트로 만드세요!)
num_features = X.select_dtypes(include=["float64", "int64","float32", "int32"]).columns.to_list()
cat_features = ["ocean_proximity"]

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [23]:
X_train_num = X_train[num_features]
X_train_cat = X_train[cat_features]
X_test_num = X_test[num_features]
X_test_cat = X_test[cat_features]

In [42]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_num_scaled = scaler.fit_transform(X_train_num)
x_test_num_scaled = scaler.transform(X_test_num)

In [43]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
X_train_cat_encoder = encoder.fit_transform(X_train_cat).toarray()
X_test_cat_encoder = encoder.transform(X_test_cat).toarray()

In [44]:
X_train_processed = np.hstack([x_train_num_scaled, X_train_cat_encoder])
X_test_processed = np.hstack([x_test_num_scaled, X_test_cat_encoder])

## 모델링

In [45]:
from sklearn.metrics import r2_score, mean_squared_error

def evaluate_model(name, y_true, y_pred) :
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    print(f"{name}")
    print(f"-- r2 : {r2:.2f}")
    print(f"-- RMSE : {rmse:.2f}")
    return {"model":name, "r2":r2, "rsme": rmse }

In [46]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_processed, y_train)
lr_ypred = lr.predict(X_test_processed)

result_lr = evaluate_model("Linear Regression", y_test, lr_ypred)
result_lr

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [41]:
# 범주형은 제외해야 합니다. 수치형만 가능해요!
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True) # 절편이 필요없는 이유 : Linear에서 자동으로 사용하기 때문
X_train_num_poly = poly.fit_transform(x_train_num_scaled) 
X_test_num_poly = poly.fit_transform(x_test_num_scaled) 

ValueError: Input X contains NaN.
PolynomialFeatures does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [52]:
X_train_poly_processed = np.hstack([X_train_num_poly, X_train_cat_encoder])
X_test_poly_processed = np.hstack([X_test_num_poly, X_test_cat_encoder])

NameError: name 'X_test_num_poly' is not defined